### Install dependencies

In [2]:
!pip install tensorflow
!pip install gym
!pip install keras
!pip install keras-rl2

### Test random environment with OpenAI Gym

In [1]:
import gym
import random
import tensorflow as tf

In [2]:
print(tf.__version__)

2.6.0


In [3]:
env = gym.make('CartPole-v0') # build cartpol environment using `make()`
states = env.observation_space.shape[0] # available states in our environment
actions = env.action_space.n # available actions in our environment

In [4]:
episodes = 10
for episode in range(1, episodes+1):
  state = env.reset()  # reset the environment after every turn
  done = False
  score = 0

  while not done:
    env.render()  # start rendering our environment
    action = random.choice([0, 1])  # takes random step out of available steps in our environment
    n_state, reward, done, info = env.step(action)  # apply action to our environment
    score+=reward
  print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:16.0
Episode:2 Score:16.0
Episode:3 Score:58.0
Episode:4 Score:15.0
Episode:5 Score:12.0
Episode:6 Score:19.0
Episode:7 Score:42.0
Episode:8 Score:18.0
Episode:9 Score:15.0
Episode:10 Score:31.0


### create a deep learning model with Keras

In [12]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [13]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1,states)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [14]:
model = build_model(states, actions)

In [15]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 4)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 24)                120       
_________________________________________________________________
dense_4 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 50        
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


### Build Agent with Keras-Rl

In [16]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [17]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [18]:
dqn = build_agent(model, actions)
dqn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
            metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 11:39 - reward: 1.0000

F:\assessment\assessment\lib\site-packages\keras\engine\training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
F:\assessment\assessment\lib\site-packages\rl\memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 104s 10ms/step - reward: 1.0000
110 episodes - episode_reward: 89.682 [9.000, 200.000] - loss: 2.553 - mae: 18.341 - mean_q: 37.079

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 111s 11ms/step - reward: 1.0000
54 episodes - episode_reward: 187.019 [154.000, 200.000] - loss: 5.232 - mae: 37.965 - mean_q: 76.580

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 118s 12ms/step - reward: 1.0000
51 episodes - episode_reward: 193.157 [162.000, 200.000] - loss: 4.274 - mae: 39.995 - mean_q: 80.462

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 117s 12ms/step - reward: 1.0000
51 episodes - episode_reward: 196.020 [161.000, 200.000] - loss: 3.773 - mae: 38.853 - mean_q: 78.088

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 112s 11ms/step - reward: 1.0000
done, took 562.412 seconds


In [19]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
Episode 11: reward: 200.000, steps: 200
Episode 12: reward: 200.000, steps: 200
Episode 13: reward: 200.000, steps: 200
Episode 14: reward: 200.000, steps: 200
Episode 15: reward: 200.000, steps: 200
Episode 16: reward: 200.000, steps: 200
Episode 17: reward: 200.000, steps: 200
Episode 18: reward: 200.000, steps: 200
Episode 19: reward: 200.000, steps: 200
Episode 20: reward: 200.000, steps: 200
Episode 21: reward: 200.000, steps: 200
Episode 22: reward: 200.000, steps: 200
Episode 23: reward: 200.000, steps: 200
Episode 24: reward: 200.000, steps: 200
Episode 25: reward: 

In [20]:
_ = dqn.test(env, nb_episodes=15, visualize=True)

Testing for 15 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
Episode 11: reward: 200.000, steps: 200
Episode 12: reward: 200.000, steps: 200
Episode 13: reward: 195.000, steps: 195
Episode 14: reward: 200.000, steps: 200
Episode 15: reward: 200.000, steps: 200


### Saving Weights

In [21]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)